In [1]:
import pandas as pd
import math
import time

In [2]:
# feature selection type variables
FORWARD_SELECTION, BACKWARD_ELIMINATION = 'forward_selection', 'backward_elimination'

# cache to reduce the computational cost during measurement of euclidean distance
cache = {}
listbox = None

In [3]:
# given a file path of the dataset, method reads dataset
# and returns all the instances with their labels
# and feature count per instance
def read_dataset(dataset_path):

    data = pd.read_csv(dataset_path, delim_whitespace=True, header = None)
    instance_count, column_count = data.shape   
    feature_count = column_count - 1
    instances = data.values.tolist()
    
    return instances, instance_count, feature_count

In [4]:
# given two instances of the dataset, and a list of features to be considered
# method find the euclidean distance based on the features given
# the method uses a global cache to reduce the computational cost incurred
# to find the distance, whenever it finds two features, that are already stored
# in the map with its squared difference, the method gets the value from the map
# otherwise, it calculates the value and stores in the map for further usage
def find_euclidean_distance(instance, compare_instance, features):
    
    squares = 0
    # iterates over the given features
    for feature in features:
        
        x = instance[feature]
        y = compare_instance[feature]
        val = 0
        
        # searching if already the calculated value is in the cache
        if (x,y) in cache:
            val = cache[(x, y)]
        elif (y,x) in cache:
            val = cache[(y, x)]
        else:
            diff = x - y
            val = diff ** 2
            
            cache[(x, y)] = val
            
        squares += val
    
    return math.sqrt(squares)

In [5]:
# given the dataset and feature set to be considered
# method predicts the label of the instance and compares the predicted one with
# the given label using nearest neighbor classifier where k = 1
# method returns the ratio of count of correctly predicted instances and
# total instances
def find_nearest_neighbor(dataset, feature_set, instance_count):
    
    correct_prediction = 0
    # iterating over each of the instance
    for instance_idx in range(instance_count):
        
        # extracts the current instance
        instance = dataset[instance_idx]
        # extracts the label of the current instance
        target = instance[0]
        # the set of features to be considered
        features = feature_set 
        
        nearest_neighbor_distance = math.inf
        nearest_neighbor_predict = -1
        
        # iterates over each of the other instances in the dataset
        # to find which instance is more close in respect of euclidean 
        # distance
        for compare_idx in range(instance_count):
            
            if compare_idx != instance_idx:
                
                compare_instance = dataset[compare_idx]
                compare_target = compare_instance[0]
                distance = find_euclidean_distance(instance,
                                                  compare_instance,
                                                  features)
                
                if distance < nearest_neighbor_distance:
                        nearest_neighbor_distance = distance
                        nearest_neighbor_predict = compare_target
        
        # checking whether the predicted label matches the target label
        if nearest_neighbor_predict == target:
            correct_prediction += 1

    # returns the ratio of the count of correctly predicted instance and 
    # total instances    
    return correct_prediction / instance_count

In [6]:
import matplotlib.pyplot as plt

# plots a graph to show how accuracy gets affected by feature counts
def plot_graph(accuracy_map, search_type, dataset_size):
    
    x = []
    y = []
    for key in sorted(accuracy_map):
        x.append(key)
        y.append(accuracy_map[key])
        
    plt.plot(x, y)
    plt.xlabel('Feature Count')
    plt.ylabel('Accuracy')
    plt.title('' + search_type.upper() + ' Dataset ' + dataset_size)
    plt.show()

In [7]:
# given the instances and feature set to be considered,
# method iteratively leaves one instance, runs nearest neighbor classifier on
# the remaining instances, and finds the accuracy, that is how, the number of 
# validation set will be equal to the number of instance count
# returns the average of the accuracy over all the validation sets
def leave_one_out_cross_validation(instances, instance_count, feature_set):
    
    accuracy_list = []
    # iterates over each of the instances
    for k_fold_itr in range(0, instance_count):
        
        # prepares a dataset removing k_fold_itr th instance
        dataset = instances[ :k_fold_itr] + instances[k_fold_itr + 1: ]

        # runs nearest neighbor classifier to finds accuracy 
        accuracy = find_nearest_neighbor(dataset, feature_set, \
                                         instance_count - 1)
        
        # appendds the accuracy in a list
        accuracy_list.append(accuracy)
    
    # returns the average accuracy found
    return sum(accuracy_list)/instance_count

In [8]:
# given the type of feature selection type along with all the instances
# iterates over different sets of features (based on the type of feature 
# selection given)
# for each different set of feature, finds leave-one-out validation accuracy
# using nearest neighbor classifier (k = 1)
# prints the best accuracy with the best feature set
# returns the best accuracy to best feature set per level to plot graph 
def start_experiment(instances, instance_count, feature_count, \
                     search_type = FORWARD_SELECTION):
    
    print()
    print()
    print(search_type.upper())
    accuracy_map = {}
    
    all_feature_set = list(range(1, feature_count + 1))
    accuracy = leave_one_out_cross_validation(\
                    instances, instance_count, all_feature_set)
    print('Running nearest neighbor with all', feature_count, \
          'features, using leave-one-out evaluation',
         'with accuracy', '{:0.1f}%'.format(accuracy * 100))
    accuracy_map[len(all_feature_set)] = accuracy
    
    # starts with an empty feature set for forward selection
    # starts with all features for backward elimination
    if search_type == FORWARD_SELECTION:
        current_feature_set = []
    elif search_type == BACKWARD_ELIMINATION:
        current_feature_set = all_feature_set
    
    print()
    print('Beginning Search ')
    best_accuracy_feature_set = all_feature_set
    best_accuracy = accuracy
    
    # iterating over each of the level
    # on each level, either the count of features
    # equal to  (level count) (forward selection)
    # or (total feature count - level count) (backward elimination)
    for level in range(1, feature_count):
        
        print('On the ', str(level), 'th level of the search tree')
        level_wise_best_accuracy_feature_set = None
        level_wise_best_accuracy_feature = None
        level_wise_best_accuracy = 0
        
        # iterating over each of the features which can be added
        # or removed for feature selection/backward elimination
        # respectively
        for feature in range(1, feature_count + 1):
            
            if (feature not in current_feature_set and \
                search_type == FORWARD_SELECTION) or \
                (feature in current_feature_set and \
                 search_type == BACKWARD_ELIMINATION):
                
                if search_type == FORWARD_SELECTION:
                    feature_set = current_feature_set + [feature]
                elif search_type == BACKWARD_ELIMINATION:
                    feature_set = list(set(current_feature_set) - set([feature]))
                
                # finding leave one out cross validation accuracy
                accuracy = leave_one_out_cross_validation(\
                                instances, instance_count, \
                                feature_set)
                
                # sorting for displaying results in good manner
                feature_set.sort()
                feature_set_string = '{' + ','.join(str(f) \
                                                    for f in feature_set) + '}'
                print('\t Using features',  feature_set_string, 'accuracy is', \
                      '{:0.1f}%'.format(accuracy * 100) )
                
                # checks if current feature set provides the better accuracy
                # among the feature sets in the current level
                if accuracy > level_wise_best_accuracy:
                    level_wise_best_accuracy = accuracy
                    level_wise_best_accuracy_feature = feature
                    level_wise_best_accuracy_feature_set = feature_set
                    
        if search_type == FORWARD_SELECTION:
            current_feature_set.append(level_wise_best_accuracy_feature)
            add_remove_log = 'adding ' + 'feature ' + \
                str(level_wise_best_accuracy_feature)
        elif search_type == BACKWARD_ELIMINATION:
            current_feature_set = list(set(current_feature_set) - \
                                        set([level_wise_best_accuracy_feature]))
            add_remove_log = 'removing ' + 'feature ' + \
                              str(level_wise_best_accuracy_feature)
            
        
        level_wise_best_feature_set_string = '{' + ','.join(str(f) \
                            for f in level_wise_best_accuracy_feature_set) + '}'
        print('Feature set ', level_wise_best_feature_set_string, \
              ' was best. accuracy is', '{:0.1f}%'.format(\
                                level_wise_best_accuracy*100), add_remove_log )
        
        accuracy_map[len(level_wise_best_accuracy_feature_set)] = \
                                level_wise_best_accuracy
        print()
        
        # checks if current level accuracy is best among the so far
        # feature sets considered
        if level_wise_best_accuracy > best_accuracy:
            best_accuracy = level_wise_best_accuracy
            best_accuracy_feature_set = level_wise_best_accuracy_feature_set
    
    # displays the best accuracy calculated with the best feature set
    best_accuracy_feature_set_string = '{' + ','.join(str(f) \
                                for f in best_accuracy_feature_set) + '}'
    print('Finished search !!', 'The best feature subset is', \
                                best_accuracy_feature_set_string, \
             'which has an accuracy of', '{:0.1f}%'.format(best_accuracy*100) )
    
    return accuracy_map
    

In [ ]:
root = None
scrollbar = None

In [122]:
import tkinter as tk


# driver function
if __name__ == "__main__":
    
    root = tk.Tk()
    root.title("Feature Selection with Nearest Neighbor")
    root.geometry('500x500')

        
    tk.Label(root, 
             text="""Choose Dataset""",
             justify = tk.LEFT,
             padx = 20
            ).place(x = 25, y = 10) 

    

    datasets = [("Small Dataset", 'small', 1),
                 ("Large Dataset", 'large', 2)]
    dataset_id = tk.StringVar()
    dataset_id.set('small')  
    
    def GetDataset():
        print(dataset_id.get())
        
    for dataset, val, pos in datasets:
        tk.Radiobutton(root, 
                       text=dataset,
                       padx = 20, 
                       variable=dataset_id, 
                       command=GetDataset,
                       value=val).place(x = 5, y = 10 + pos*20) 
    
    
    tk.Label(root, 
             text="""Choose Feature selection method""",
             justify = tk.LEFT,
             padx = 20
            ).place(x = 25, y = 90) 
    
    methods = [("Forward Selection", 'forward', 1),
                 ("Backward Elimination", 'backward', 2)
                ]
    
    method_id = tk.StringVar()
    method_id.set('forward')  
    
    def GetMethod():
        print(method_id.get())
        
    for method, val, pos in methods:
        tk.Radiobutton(root, 
                       text=method,
                       padx = 20, 
                       variable=method_id, 
                       command=GetMethod,
                       value=val).place(x = 5, y = 90 + pos*20)
        
    
    tk.Label(root, 
             text="""Choose validation type""",
             justify = tk.LEFT,
             padx = 20
            ).place(x = 25, y = 160) 
    
    validations = [("Leave-one-out Validation", 'leave-one-out', 1),
                 ("10-Fold Vaildation", '10-fold', 2)
                ]
    
    validation_id = tk.StringVar()
    validation_id.set('leave-one-out')  
    
    def GetValidation():
        print(validation_id.get())
        
    for validation, val, pos in validations:
        tk.Radiobutton(root, 
                       text=validation,
                       padx = 20, 
                       variable=validation_id, 
                       command=GetValidation,
                       value=val).place(x = 5, y = 160 + pos*20)
    
    # Creating a Scrollbar and 
    # attaching it to root window
    scrollbar = Scrollbar(root)

    # Adding Scrollbar to the right
    # side of root window
    scrollbar.pack(side = RIGHT, fill = BOTH)
    
    # setting scrollbar command parameter 
    # to listbox.yview method its yview because
    # we need to have a vertical view
    scrollbar.config(command = listbox.yview)    
    
    def start_experiment_with_gui():
        
        start_time = time.time()
        if dataset_id.get() == 'small' and method_id.get() == 'forward':
            listbox.insert(END,"Forward Selection selected with small dataset")
            instances, instance_count, feature_count = read_dataset(small_dataset_path)
            
            accuracy_map = start_experiment(instances, instance_count, \
                                            feature_count, FORWARD_SELECTION)
            plot_graph(accuracy_map, FORWARD_SELECTION, 'small')
                
        elif dataset_id.get() == 'large' and method_id.get() == 'forward':
            listbox.insert(END,"Forward Selection selected with large dataset")
            instances, instance_count, feature_count = read_dataset(large_dataset_path)
            
            accuracy_map = start_experiment(instances, instance_count, \
                                            feature_count, FORWARD_SELECTION)
            plot_graph(accuracy_map, FORWARD_SELECTION, 'large')
            
            
        elif dataset_id.get() == 'small' and method_id.get() == 'backward':
            listbox.insert(END,'Backward Elimination selected with small dataset')
            instances, instance_count, feature_count = read_dataset(small_dataset_path)
            
            accuracy_map = start_experiment(instances, instance_count, \
                                            feature_count, BACKWARD_ELIMINATION)
            plot_graph(accuracy_map, BACKWARD_ELIMINATION, 'small')
            
            
        elif dataset_id.get() == 'large' and method_id.get() == 'backward':
            listbox.insert(END,'Backward Elimination selected with large dataset')
            
            instances, instance_count, feature_count = read_dataset(large_dataset_path)
            
            accuracy_map = start_experiment(instances, instance_count, \
                                            feature_count, BACKWARD_ELIMINATION)
            plot_graph(accuracy_map, BACKWARD_ELIMINATION, 'large')
            
        end_time = time.time()
        execution_time_in_seconds = end_time - start_time
        listbox.insert(END,'Execution time ', '{:0.1f} seconds'.format(execution_time_in_seconds))
        
        print()
        print()
    
    def find_features():
        root.after(10000, start_experiment_with_gui)
    
    tk.Button(root, text= "Find Features", \
              command=find_features).place(x = 5, y = 230)
    
    lbl = tk.Label(root, text='', font=("Helvetica", "24"))
    lbl.place(x = 5, y = 260) 

    listbox = Listbox(lbl, width = 50, height = 50)


    # Adding Listbox to the left
    # side of root window
    listbox.pack(side = LEFT, fill = BOTH)


    # Insert elements into the listbox
    for values in range(100):
        listbox.insert(END, values)

    # Attaching Listbox to Scrollbar
    # Since we need to have a vertical 
    # scroll we use yscrollcommand
    listbox.config(yscrollcommand = scrollbar.set)
    listbox.configure(background="black", foreground="white")

#     listbox.insert(END, 100)
#     listbox.pack_forget()
    
#     lbl.pack(anchor="w")
    
    
    
    root.mainloop()



FORWARD_SELECTION


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/masud/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/Users/masud/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 814, in callit
    func(*args)
  File "<ipython-input-122-a8138e99ffa6>", line 106, in start_experiment_with_gui
    accuracy_map = start_experiment(instances, instance_count, \
  File "<ipython-input-8-cf00986bf3b2>", line 17, in start_experiment
    accuracy = leave_one_out_cross_validation(\
  File "<ipython-input-7-0979ffb986d1>", line 16, in leave_one_out_cross_validation
    accuracy = find_nearest_neighbor(dataset, feature_set, \
  File "<ipython-input-5-fe7c540b8530>", line 31, in find_nearest_neighbor
    distance = find_euclidean_distance(instance,
  File "<ipython-input-4-ae8173e6d8ce>", line 20, in find_euclidean_distance
    elif (y,x) in cache:
KeyboardInterrupt


KeyboardInterrupt: 

In [76]:
# driver function
if __name__ == "__main__":
    
#     small_dataset_path = 'CS205_CalibrationData__1.txt'
    
    small_dataset_path = 'CS205_SP_2022_SMALLtestdata__35.txt'
    large_dataset_path = 'CS205_SP_2022_Largetestdata__62.txt'
    
    while True:

        input_case = input('Press 11 to run Forward Selection with small dataset\n' + \
                          'Press 12 to run Forward Selection with large dataset\n' + \
                          'Press 21 to run Backward Elimination with small dataset\n' + \
                          'Press 22 to run Backward Elimination with large dataset\n' + \
                          'Press any other key to exit\n').strip()
        
        start_time = time.time()
        
        if input_case == '11':
            print("Forward Selection selected with small dataset")
            instances, instance_count, feature_count = read_dataset(small_dataset_path)
            
            accuracy_map = start_experiment(instances, instance_count, \
                                            feature_count, FORWARD_SELECTION)
            plot_graph(accuracy_map, FORWARD_SELECTION, 'small')
                
        elif input_case == '12':
            print("Forward Selection selected with large dataset")
            instances, instance_count, feature_count = read_dataset(large_dataset_path)
            
            accuracy_map = start_experiment(instances, instance_count, \
                                            feature_count, FORWARD_SELECTION)
            plot_graph(accuracy_map, FORWARD_SELECTION, 'large')
            
            
        elif input_case == '21':
            print('Backward Elimination selected with small dataset')
            instances, instance_count, feature_count = read_dataset(small_dataset_path)
            
            accuracy_map = start_experiment(instances, instance_count, \
                                            feature_count, BACKWARD_ELIMINATION)
            plot_graph(accuracy_map, BACKWARD_ELIMINATION, 'small')
            
            
        elif input_case == '22':
            print('Backward Elimination selected with large dataset')
            
            instances, instance_count, feature_count = read_dataset(large_dataset_path)
            
            accuracy_map = start_experiment(instances, instance_count, \
                                            feature_count, BACKWARD_ELIMINATION)
            plot_graph(accuracy_map, BACKWARD_ELIMINATION, 'large')
            
            
        else:
            print('Exit. Thank you')
            break
        
        end_time = time.time()
        execution_time_in_seconds = end_time - start_time
        print('Execution time ', '{:0.1f} seconds'.format(execution_time_in_seconds))
        
        print()
        print()

KeyboardInterrupt: Interrupted by user